# Day 7: Accessibility

Let's create a map with travel times to access public transportation, identifying where park & ride is an option for people with cars.

In [ ]:
from sedona.spark import *
import os

config = SedonaContext.builder().getOrCreate()
sedona = SedonaContext.create(config)
user_path = os.getenv("USER_S3_PATH")

## Get list of categories and find the transit-related ones

Query the Wherobots Open Data hosted Overture Maps Foundation places_place table to get all the primary categories, then export them to CSV to have Gemini help classiy them by likelihood of being transit-related.

In [2]:
# Get list of all primary categories in Overture
categories_df = sedona.sql("SELECT categories.primary as primary_cat, count(*) as cnt FROM wherobots_open_data.overture_maps_foundation.places_place GROUP BY primary_cat")
categories_df.count()

# Write to a single CSV file by forcing the operation to run on 1 exeucutor
categories_df.coalesce(1).write.mode("overwrite").option("header", "true").csv(f"{user_path}OMF_categories")

## Lightly clean the category-based results

I want to get the categories that are used for public transit. Gemini helped me find the likely categories from the CSV file. I reviewed its list of high-likelihood categories, took out one category and added a few others. I live in near Seattle and I'm familiar with Paris, so looked at the results from that query to find some categories that are not public transit.

In [3]:
import wkls

transit_test_df = sedona.sql('''
SELECT
    id,
    names.primary as name,
    basic_category,
    categories,
    geometry,
    addresses[0] as address
FROM wherobots_open_data.overture_maps_foundation.places_place
WHERE
    categories.primary IN (
        'metro_station', 'ferry_service', 'trains' ,'bike_sharing', 'transportation', 'public_transportation',
        'park_and_rides', 'car_sharing', 'railway_service', 'bicycle_sharing_location', 'light_rail_and_subway_stations',
        'train_station', 'bus_station', 'cable_car_service', 'transport_interchange', 'water_taxi'
    )
    AND
    (
        ST_Within(geometry, ST_GeomFromWKT("''' + wkls.us.wa.seattle.wkt() + '''")) OR
        ST_Within(geometry, ST_GeomFromWKT("''' + wkls.fr.idf.paris.wkt() + '''"))
    )
''')

transit_test_df.show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                  id|                name|      basic_category|          categories|            geometry|             address|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|64144b3d-7b15-4ab...| Eagle Cabulance LLC|transportation_lo...|{transportation, ...|POINT (-122.27014...|{8105 Rainier Ave...|
|f47d68b9-35dc-4ba...|Link Light Rail d...|transportation_lo...|{transportation, ...|POINT (-122.27510...|{NULL, Seattle, N...|
|f696a827-183b-41f...|Link Light Rail-O...|transportation_lo...|{transportation, ...|POINT (-122.28175...|{7100 Martin Luth...|
|af41a50f-3bc5-424...|     Othello Station|       train_station|{train_station, [...|POINT (-122.28152...|{7100 Martin Luth...|
|e674c4ca-a514-4f3...|Rainier Beach sta...|       train_station|{train_station, [...|POINT (-122.27936..

Here's the final query, with global results and some alternate categories filtered out. It's not a perfect method, but it does a reasonable job cleaning the data a bit.

There are other optimizations I would do if this were mission-critical analytics, like removing items with very similar names and locations (like the three entries for "Saint-Michel – Notre-Dame," for example).

In [4]:
transit_global_df = sedona.sql('''
SELECT
    id,
    names.primary as name,
    basic_category,
    categories,
    geometry,
    addresses[0] as address
FROM wherobots_open_data.overture_maps_foundation.places_place
WHERE
    categories.primary IN (
        'metro_station', 'ferry_service', 'trains' ,'bike_sharing', 'transportation', 'public_transportation',
        'park_and_rides', 'car_sharing', 'railway_service', 'bicycle_sharing_location', 'light_rail_and_subway_stations',
        'train_station', 'bus_station', 'cable_car_service', 'transport_interchange', 'water_taxi'
    )
    AND NOT arrays_overlap(
        categories.alternate, 
        array(
            'travel_services', 'tours', 'travel', 'contractor', 'ferry_boat_company', 'freight_and_cargo_service', 
            'b2b_science_and_technology', 'limo_services', 'professional_services', 'pharmacy', 'restaurant', 'boat_tours',
            'bar', 'public_plaza', 'monument', 'rental_service', 'ambulance_and_ems_services', 'janitorial_services',
            'event_planning', 'grocery_store', 'car_rental_agency', 'shipping_center', 'airline', 'airport_shuttles',
            'catholic_church'
        )
    )
''')

transit_global_df.count()

126887

## Determinining travel times

As a (not ideal) proxy for walkability, I used a 3-minute drive time to see where there is access to public transit everywhere OMF has data. I wrote it to a single GeoParquet file so it could be easily imported to [Kepler.gl](https://kepler.gl) or other visualization tools.

In [5]:
transit_global_df = transit_global_df.createOrReplaceTempView("transit_global_df")

isochrones_df = sedona.sql('''SELECT *, ST_Isochrone(geometry, 3, 'car', false) AS isochrone FROM transit_global_df''').cache()
isochrones_df.coalesce(1).write.mode("overwrite").format("geoparquet").save(f"{user_path}Transit_Isochrones_3min.parquet")

isochrones_df.count()

126887

As an option to using a visualization tool, I also generated a PMTile set of the vectors. There's a preview below of the tiles rendered using Leaflet.

In [6]:
from wherobots import vtiles
from pyspark.sql.functions import col, lit

isochrones_tiles_df = isochrones_df. \
    drop("id", "basic_category", "categories", "geometry", "address"). \
    withColumn("layer", lit("Isochrone_3_minute_drive")). \
    withColumn("geometry", col("isochrone")). \
    drop("isochrone"). \
    filter("NOT ST_IsEmpty(geometry)")

isochrones_tiles_df.show()
vtiles.generate_pmtiles(isochrones_tiles_df, f"{user_path}global_transit_isochrones.pmtiles")

+--------------------+--------------------+--------------------+
|                name|               layer|            geometry|
+--------------------+--------------------+--------------------+
|San Cristóbal -Mu...|Isochrone_3_minut...|POLYGON ((-89.617...|
|Taxi Express Gala...|Isochrone_3_minut...|POLYGON ((-90.328...|
|Alya Galapagos Cr...|Isochrone_3_minut...|POLYGON ((-90.328...|
|Naviera Easter Is...|Isochrone_3_minut...|POLYGON ((-109.44...|
| Eastern Bus Station|Isochrone_3_minut...|POLYGON ((-175.21...|
| Western Bus Station|Isochrone_3_minut...|POLYGON ((-175.21...|
|         Silva Group|Isochrone_3_minut...|POLYGON ((-171.79...|
|   Temana Assistance|Isochrone_3_minut...|POLYGON ((-149.89...|
|    Pacific Princess|Isochrone_3_minut...|POLYGON ((-149.57...|
|Gare Maritime De ...|Isochrone_3_minut...|POLYGON ((-149.58...|
|Papenoo Hydroelec...|Isochrone_3_minut...|POLYGON ((-149.43...|
|        EURL Poerava|Isochrone_3_minut...|POLYGON ((-151.76...|
|  Empresarios Unidos|Iso

In [7]:
vtiles.show_pmtiles(f"{user_path}global_transit_isochrones.pmtiles")

Finally, a preview of the map as a static image: 

![Transit access map](../images/day07_jp_travel_isochrones.png)